# Applied Data Science Capstone Project - The Battle of Neighborhood Part - 2

## Data Preparation

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import numpy as np
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


**Get PostalCode and Geoinfo**

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/zauberware/postal-codes-json-xml-csv/master/data/DE/zipcodes.de.csv') #geting geocode from mentioned csv
df.head()

,country_code,zipcode,place,state,state_code,province,province_code,community,community_code,latitude,longitude
0,DE,1945,Grünewald,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4000,14.0000
1,DE,1945,Lindenau,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4000,13.7333
2,DE,1945,Hohenbocka,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4310,14.0098
3,DE,1945,Schwarzbach,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4500,13.9333
4,DE,1945,Guteborn,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4167,13.9333


In [21]:
munich = df[df['community'] == 'München'].reset_index(drop=True)
munich.head()

,country_code,zipcode,place,state,state_code,province,province_code,community,community_code,latitude,longitude
0,DE,80331,München,Bayern,BY,Upper Bavaria,91,München,9162,48.1345,11.5710
1,DE,80333,München,Bayern,BY,Upper Bavaria,91,München,9162,48.1452,11.5668
2,DE,80335,München,Bayern,BY,Upper Bavaria,91,München,9162,48.1427,11.5552
3,DE,80336,München,Bayern,BY,Upper Bavaria,91,München,9162,48.1345,11.5590
4,DE,80337,München,Bayern,BY,Upper Bavaria,91,München,9162,48.1224,11.5449


In [24]:
munich.shape

(73, 11)

**Get Munich District data**

In [6]:
url = 'https://www.muenchen.de/int/en/living/postal-codes.html'
munich_post_list = pd.read_html(url)
munich_post = munich_post_list[0]
munich_post

,District,Postal Code
0,Allach-Untermenzing,"80995, 80997, 80999, 81247, 81249"
1,Altstadt-Lehel,"80331, 80333, 80335, 80336, 80469, 80538, 80539"
2,Au-Haidhausen,"81541, 81543, 81667, 81669, 81671, 81675, 81677"
3,Aubing-Lochhausen-Langwied,"81243, 81245, 81249"
4,Berg am Laim,"81671, 81673, 81735, 81825"
5,Bogenhausen,"81675, 81677, 81679, 81925, 81927, 81929"
6,Feldmoching-Hasenbergl,"80933, 80935, 80995"
7,Hadern,"80689, 81375, 81377"
8,Laim,"80686, 80687, 80689"
9,Ludwigsvorstadt-Isarvorstadt,"80335, 80336, 80337, 80469"


In [7]:
#Combine data together to get Geo data by district
muc = pd.DataFrame(columns=['District', 'Postal Code'])
muc.head()

,District,Postal Code


In [9]:
items = []
for idx, codes in enumerate(munich_post['Postal Code']):
    code_list = codes.split(',')
    district = munich_post['District'][idx]
    for element in code_list:
        element = element.replace(' ', '')
        items.append({'District': district, 'Postal Code': element})

In [10]:
muc = muc.append(items)
muc.head()

,District,Postal Code
0,Allach-Untermenzing,80995
1,Allach-Untermenzing,80997
2,Allach-Untermenzing,80999
3,Allach-Untermenzing,81247
4,Allach-Untermenzing,81249


In [12]:
latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array
postal_codes = muc['Postal Code']

In [25]:
#Mapping postal code between two data frames
for postal_code in postal_codes : 
    lat = df[df['zipcode']== int(postal_code)]['latitude'].iloc[0]
    lon = df[df['zipcode'] == int(postal_code)]['longitude'].iloc[0]
    
    latitudes.append(lat)
    longitudes.append(lon)

In [26]:
muc['Latitude'] = latitudes
muc['Longitude'] = longitudes

In [27]:
muc.head()

,District,Postal Code,Latitude,Longitude
0,Allach-Untermenzing,80995,48.1976,11.5181
1,Allach-Untermenzing,80997,48.1834,11.4784
2,Allach-Untermenzing,80999,48.1853,11.4643
3,Allach-Untermenzing,81247,48.1662,11.4673
4,Allach-Untermenzing,81249,48.1500,11.5833


## Get Forsquare API

In [76]:
CLIENT_ID = '*****' # your Foursquare ID
CLIENT_SECRET = '*****' # your Foursquare Secret
ACCESS_TOKEN = '*****' # your FourSquare Access Token {"access_token":"PG1GWWTYRRMWEGQZ3AU4NHZINGSYCY1RK3UAWUR4MW3ZKDVV"}
VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: *****
CLIENT_SECRET:*****


## Analyse venues around Munich Main Train Station

In [77]:

latitude = 48.14049 
longitude = 11.55783
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=*****&client_secret=*****&ll=48.14049,11.55783&v=20180604&radius=500&limit=30'

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
results = requests.get(url).json()
'There are {} around .'.format(len(results['response']['groups'][0]['items']))
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b8ffa3ff964a520a16d33e3',
  'name': 'Sofitel Munich Bayerpost',
  'location': {'address': 'Bayerstrasse 12',
   'lat': 48.13945940218924,
   'lng': 11.556781339683198,
   'labeledLatLngs': [{'label': 'display',
     'lat': 48.13945940218924,
     'lng': 11.556781339683198}],
   'distance': 138,
   'postalCode': '80335',
   'cc': 'DE',
   'city': 'München',
   'state': 'Bayern',
   'country': 'Deutschland',
   'formattedAddress': ['Bayerstrasse 12', '80335 München', 'Deutschland']},
  'categories': [{'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'shortName': 'Hotel',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4b8ffa3ff964a520a16d33e3-0'}

In [33]:
#Display by data frame
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Sofitel Munich Bayerpost,Hotel,Bayerstrasse 12,DE,München,Deutschland,NaN,138,"[Bayerstrasse 12, 80335 München, Deutschland]","[{'label': 'display', 'lat': 48.13945940218924...",48.139459,11.556781,NaN,80335,Bayern,4b8ffa3ff964a520a16d33e3
1,ISA.BAR,Hotel Bar,Bayerstr. 12,DE,München,Deutschland,NaN,140,"[Bayerstr. 12, 80335 München, Deutschland]","[{'label': 'display', 'lat': 48.13924788044313...",48.139248,11.557481,NaN,80335,Bayern,4ef257f693adff223e5d77b5
2,25hours Hotel The Royal Bavarian,Hotel,Bahnhofplatz 1,DE,München,Deutschland,NaN,277,"[Bahnhofplatz 1, 80335 München, Deutschland]","[{'label': 'display', 'lat': 48.13972312843083...",48.139723,11.561390,NaN,80335,Bayern,59b2cd393c858d7f3b7d0dde
3,L'Osteria,Trattoria/Osteria,Arnulfstrasse 1,DE,München,Deutschland,NaN,136,"[Arnulfstrasse 1, 80335 München, Deutschland]","[{'label': 'display', 'lat': 48.14105352219812...",48.141054,11.559457,NaN,80335,Bayern,565b7dbd498e91d78d23fc40
4,Peter & Wolff,Modern European Restaurant,NaN,DE,München,Deutschland,NaN,165,"[München, Deutschland]","[{'label': 'display', 'lat': 48.14155257501139...",48.141553,11.559388,NaN,NaN,Bayern,561008e9498eed61033e15dd
5,Hofpfisterei,Bakery,Arnulfstr. 1,DE,München,Deutschland,S-Bahn Sperrengeschoss,225,"[Arnulfstr. 1 (S-Bahn Sperrengeschoss), 80335 ...","[{'label': 'display', 'lat': 48.14107637044532...",48.141076,11.560741,NaN,80335,Bayern,4c0faf522466a5934bff7a21
6,Irmi,Bavarian Restaurant,NaN,DE,NaN,Deutschland,NaN,195,"[80336, Deutschland]","[{'label': 'display', 'lat': 48.138739, 'lng':...",48.138739,11.558064,NaN,80336,NaN,5bc2fa7686f4cc002c720b8c
7,Café Kosmos,Bar,Dachauer Str. 7,DE,München,Deutschland,Marsstr.,276,"[Dachauer Str. 7 (Marsstr.), 80335 München, De...","[{'label': 'display', 'lat': 48.142484, 'lng':...",48.142484,11.560045,NaN,80335,Bayern,4b12fb1cf964a5200f9223e3
8,marc münchen,Hotel,Senefelderstr. 12,DE,München,Deutschland,NaN,312,"[Senefelderstr. 12, 80336 München, Deutschland]","[{'label': 'display', 'lat': 48.13803204118574...",48.138032,11.559863,NaN,80336,Bayern,4bc8f29d762beee16fbb3d38
9,Fleming's Brasserie & Delikatessen,Wine Bar,Bayerstr. 47,DE,München,Deutschland,NaN,167,"[Bayerstr. 47, 80335 München, Deutschland]","[{'label': 'display', 'lat': 48.13902441013806...",48.139024,11.557300,NaN,80335,Bayern,4e1ad617922e475a0bd91839


### Display map around Munich Hauptbahnhof

In [34]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='ISA.BAR',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [36]:
hauphbahnhof = dataframe_filtered
hauphbahnhof.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Sofitel Munich Bayerpost,Hotel,Bayerstrasse 12,DE,München,Deutschland,NaN,138,"[Bayerstrasse 12, 80335 München, Deutschland]","[{'label': 'display', 'lat': 48.13945940218924...",48.139459,11.556781,NaN,80335,Bayern,4b8ffa3ff964a520a16d33e3
1,ISA.BAR,Hotel Bar,Bayerstr. 12,DE,München,Deutschland,NaN,140,"[Bayerstr. 12, 80335 München, Deutschland]","[{'label': 'display', 'lat': 48.13924788044313...",48.139248,11.557481,NaN,80335,Bayern,4ef257f693adff223e5d77b5
2,25hours Hotel The Royal Bavarian,Hotel,Bahnhofplatz 1,DE,München,Deutschland,NaN,277,"[Bahnhofplatz 1, 80335 München, Deutschland]","[{'label': 'display', 'lat': 48.13972312843083...",48.139723,11.561390,NaN,80335,Bayern,59b2cd393c858d7f3b7d0dde
3,L'Osteria,Trattoria/Osteria,Arnulfstrasse 1,DE,München,Deutschland,NaN,136,"[Arnulfstrasse 1, 80335 München, Deutschland]","[{'label': 'display', 'lat': 48.14105352219812...",48.141054,11.559457,NaN,80335,Bayern,565b7dbd498e91d78d23fc40
4,Peter & Wolff,Modern European Restaurant,NaN,DE,München,Deutschland,NaN,165,"[München, Deutschland]","[{'label': 'display', 'lat': 48.14155257501139...",48.141553,11.559388,NaN,NaN,Bayern,561008e9498eed61033e15dd


In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Analyse Venues around Munich Hauptbahnhof

In [38]:
print('There are {} uniques categories.'.format(len(hauphbahnhof['categories'].unique())))

There are 21 uniques categories.


In [40]:
# one hot encoding
bahn_onehot = pd.get_dummies(hauphbahnhof[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bahn_onehot['Neighborhood'] = hauphbahnhof['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bahn_onehot.columns[-1]] + list(bahn_onehot.columns[:-1])
bahn_onehot = bahn_onehot[fixed_columns]

bahn_onehot.head()

,Neighborhood,Bakery,Bar,Bavarian Restaurant,Brewery,Coffee Shop,German Restaurant,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Israeli Restaurant,Lounge,Mediterranean Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Sausage Shop,Steakhouse,Supermarket,Trattoria/Osteria,Wine Bar
0,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [41]:
bahn_onehot.shape

(30, 22)

In [42]:
bahn_grouped = bahn_onehot.groupby('Neighborhood').mean().reset_index()
bahn_grouped

,Neighborhood,Bakery,Bar,Bavarian Restaurant,Brewery,Coffee Shop,German Restaurant,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Israeli Restaurant,Lounge,Mediterranean Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Sausage Shop,Steakhouse,Supermarket,Trattoria/Osteria,Wine Bar
0,Ludwigsvorstadt - Isarvorstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [43]:
bahn_grouped.shape

(1, 22)

In [44]:
num_top_venues = 5

for hood in bahn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bahn_grouped[bahn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ludwigsvorstadt - Isarvorstadt----
                venue  freq
0            Wine Bar   1.0
1  Israeli Restaurant   0.0
2   Trattoria/Osteria   0.0
3         Supermarket   0.0
4          Steakhouse   0.0




### So the result in Muchen Hauptbanhof shows that the Wine Bar is the most frequently visited places.

### Now let's check how is it in general in the city of Munich

In [45]:
muc.head() #Display data

,District,Postal Code,Latitude,Longitude
0,Allach-Untermenzing,80995,48.1976,11.5181
1,Allach-Untermenzing,80997,48.1834,11.4784
2,Allach-Untermenzing,80999,48.1853,11.4643
3,Allach-Untermenzing,81247,48.1662,11.4673
4,Allach-Untermenzing,81249,48.1500,11.5833


## Display City map and the venues in Munich 

In [47]:
# Munich latitude and longitude using Google search
muc_lat = 48.69668
muc_lng = 13.46314

# Creates map of Munich using latitude and longitude values
map_munich = folium.Map(location=[muc_lat, muc_lng], zoom_start=10)

# Add markers to map
for lat, lng, borough in zip(muc['Latitude'], muc['Longitude'], muc['District']):
    label = '{}'.format( borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_munich)  
    
map_munich

In [50]:
muc_venues = getNearbyVenues(names=muc['District'],
                                   latitudes=muc['Latitude'],
                                   longitudes=muc['Longitude']
                                  ) #Display Venues in Munich, Germany

Allach-Untermenzing
Allach-Untermenzing
Allach-Untermenzing
Allach-Untermenzing
Allach-Untermenzing
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Aubing-Lochhausen-Langwied
Aubing-Lochhausen-Langwied
Aubing-Lochhausen-Langwied
Berg am Laim
Berg am Laim
Berg am Laim
Berg am Laim
Bogenhausen
Bogenhausen
Bogenhausen
Bogenhausen
Bogenhausen
Bogenhausen
Feldmoching-Hasenbergl
Feldmoching-Hasenbergl
Feldmoching-Hasenbergl
Hadern
Hadern
Hadern
Laim
Laim
Laim
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Milbertshofen-Am Hart
Milbertshofen-Am Hart
Milbertshofen-Am Hart
Milbertshofen-Am Hart
Moosach
Moosach
Moosach
Moosach
Moosach
Neuhausen-Nymphenburg
Neuhausen-Nym

In [51]:
print(muc_venues.shape)
muc_venues.head()

(2414, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allach-Untermenzing,48.1976,11.5181,Indian Village,48.197932,11.524083,Indian Restaurant
1,Allach-Untermenzing,48.1976,11.5181,EDEKA Fratzl,48.197393,11.524481,Supermarket
2,Allach-Untermenzing,48.1976,11.5181,Pizza Fasanerie,48.197816,11.524199,Pizza Place
3,Allach-Untermenzing,48.1976,11.5181,H Fasanerie Bahnhof,48.196895,11.524417,Bus Stop
4,Allach-Untermenzing,48.1834,11.4784,NORMA,48.183488,11.478840,Supermarket


## Analyse Venues in Munich, Germany

In [52]:
muc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allach-Untermenzing,58,58,58,58,58,58
Altstadt-Lehel,204,204,204,204,204,204
Au-Haidhausen,183,183,183,183,183,183
Aubing-Lochhausen-Langwied,42,42,42,42,42,42
Berg am Laim,36,36,36,36,36,36
Bogenhausen,98,98,98,98,98,98
Feldmoching-Hasenbergl,14,14,14,14,14,14
Hadern,18,18,18,18,18,18
Laim,58,58,58,58,58,58


In [53]:
print('There are {} uniques categories.'.format(len(muc_venues['Venue Category'].unique())))

There are 201 uniques categories.


In [54]:
# one hot encoding
muc_onehot = pd.get_dummies(muc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
muc_onehot['Neighborhood'] = muc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [muc_onehot.columns[-1]] + list(muc_onehot.columns[:-1])
muc_onehot = muc_onehot[fixed_columns]

muc_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Bosnian Restaurant,Botanical Garden,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Canal,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laundromat,Laundry Service,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Manti Place,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music School,Music Store,Music Venue,Nightclub,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Sandwich Place,Sausage Shop,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Tour Provider,Toy / Game Store,Track,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Allach-Untermenzing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Allach-Untermenzing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Allach-Untermenzing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [55]:
muc_onehot.shape

(2414, 202)

In [56]:
muc_grouped = muc_onehot.groupby('Neighborhood').mean().reset_index()
muc_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Bosnian Restaurant,Botanical Garden,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Canal,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laundromat,Laundry Service,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Manti Place,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music School,Music Store,Music Venue,Nightclub,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Sandwich Place,Sausage Shop,Seafood Restaurant,Shipping Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Tour Provider,Toy / Game Store,Track,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Allach-Untermenzing,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.017241,0.000000,0.000000,0.000000,0.00000,0.017241,0.068966,0.000000,0.051724,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.017241,0.068966,0.000000,0.000000,0.000000,0.086207,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.00000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.034483,0.000000,0.034483,0.017241,0.000000,0.000000,0.000000,0.000000,0.103448,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [57]:
muc_grouped.shape

(25, 202)

In [58]:
num_top_venues = 5

for hood in muc_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = muc_grouped[muc_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allach-Untermenzing----
                venue  freq
0  Italian Restaurant  0.10
1                Café  0.09
2         Supermarket  0.07
3            Bus Stop  0.07
4              Bakery  0.07


----Altstadt-Lehel----
                venue  freq
0                Café  0.11
1               Hotel  0.05
2  Italian Restaurant  0.05
3               Plaza  0.04
4                 Bar  0.03


----Au-Haidhausen----
                venue  freq
0  Italian Restaurant  0.08
1   German Restaurant  0.05
2               Plaza  0.04
3                Café  0.04
4            Bus Stop  0.03


----Aubing-Lochhausen-Langwied----
                venue  freq
0                Café  0.12
1  Italian Restaurant  0.10
2              Bakery  0.07
3                 Bar  0.07
4      Ice Cream Shop  0.05


----Berg am Laim----
                venue  freq
0         Supermarket  0.14
1              Bakery  0.11
2           Drugstore  0.08
3  Italian Restaurant  0.06
4            Bus Stop  0.06


----Bogenhausen----
 

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Check the most common venues in Munich

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = muc_grouped['Neighborhood']

for ind in np.arange(muc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(muc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allach-Untermenzing,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
1,Altstadt-Lehel,Café,Hotel,Italian Restaurant,Plaza,Bar,Middle Eastern Restaurant,Cocktail Bar,Art Museum,Burger Joint,Restaurant
2,Au-Haidhausen,Italian Restaurant,German Restaurant,Café,Plaza,Supermarket,Bar,Drugstore,Bakery,Bus Stop,Burger Joint
3,Aubing-Lochhausen-Langwied,Café,Italian Restaurant,Bar,Bakery,Ice Cream Shop,Plaza,Soccer Field,Post Office,Pastry Shop,Supermarket
4,Berg am Laim,Supermarket,Bakery,Drugstore,Hotel,Metro Station,Bus Stop,Asian Restaurant,Italian Restaurant,Soccer Field,Bavarian Restaurant


## Cluster Neighborhoods using K-Means

In [61]:
# set number of clusters
kclusters = 5

muc_grouped_clustering = muc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(muc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 0, 2, 3, 1, 0], dtype=int32)

In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels00', kmeans.labels_)

muc_merged = muc

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
muc_merged = muc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='District')

muc_merged.head() # check the last columns!

,District,Postal Code,Latitude,Longitude,Cluster Labels00,Cluster Labels01,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allach-Untermenzing,80995,48.1976,11.5181,0,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
1,Allach-Untermenzing,80997,48.1834,11.4784,0,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
2,Allach-Untermenzing,80999,48.1853,11.4643,0,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
3,Allach-Untermenzing,81247,48.1662,11.4673,0,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
4,Allach-Untermenzing,81249,48.1500,11.5833,0,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop


### Display cluster by map

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(muc_merged['Latitude'], muc_merged['Longitude'], muc_merged['District'], muc_merged['Cluster Labels00']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [68]:
muc_merged.loc[muc_merged['Cluster Labels00'] == 0, muc_merged.columns[[1] + list(range(5, muc_merged.shape[1]))]]

,Postal Code,Cluster Labels01,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,80995,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
1,80997,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
2,80999,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
3,81247,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
4,81249,0,0,Italian Restaurant,Café,Bakery,Bus Stop,Supermarket,Bar,Sporting Goods Shop,German Restaurant,Plaza,Ice Cream Shop
5,80331,0,0,Café,Hotel,Italian Restaurant,Plaza,Bar,Middle Eastern Restaurant,Cocktail Bar,Art Museum,Burger Joint,Restaurant
6,80333,0,0,Café,Hotel,Italian Restaurant,Plaza,Bar,Middle Eastern Restaurant,Cocktail Bar,Art Museum,Burger Joint,Restaurant
7,80335,0,0,Café,Hotel,Italian Restaurant,Plaza,Bar,Middle Eastern Restaurant,Cocktail Bar,Art Museum,Burger Joint,Restaurant
8,80336,0,0,Café,Hotel,Italian Restaurant,Plaza,Bar,Middle Eastern Restaurant,Cocktail Bar,Art Museum,Burger Joint,Restaurant
9,80469,0,0,Café,Hotel,Italian Restaurant,Plaza,Bar,Middle Eastern Restaurant,Cocktail Bar,Art Museum,Burger Joint,Restaurant


### Cluster 2

In [69]:
muc_merged.loc[muc_merged['Cluster Labels00'] == 1, muc_merged.columns[[1] + list(range(5, muc_merged.shape[1]))]]

,Postal Code,Cluster Labels01,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,81671,1,1,Supermarket,Bakery,Drugstore,Hotel,Metro Station,Bus Stop,Asian Restaurant,Italian Restaurant,Soccer Field,Bavarian Restaurant
23,81673,1,1,Supermarket,Bakery,Drugstore,Hotel,Metro Station,Bus Stop,Asian Restaurant,Italian Restaurant,Soccer Field,Bavarian Restaurant
24,81735,1,1,Supermarket,Bakery,Drugstore,Hotel,Metro Station,Bus Stop,Asian Restaurant,Italian Restaurant,Soccer Field,Bavarian Restaurant
25,81825,1,1,Supermarket,Bakery,Drugstore,Hotel,Metro Station,Bus Stop,Asian Restaurant,Italian Restaurant,Soccer Field,Bavarian Restaurant
38,80686,1,1,Supermarket,Bakery,Bus Stop,Doner Restaurant,Bank,Organic Grocery,Mobile Phone Shop,Restaurant,Sporting Goods Shop,Drugstore
39,80687,1,1,Supermarket,Bakery,Bus Stop,Doner Restaurant,Bank,Organic Grocery,Mobile Phone Shop,Restaurant,Sporting Goods Shop,Drugstore
40,80689,1,1,Supermarket,Bakery,Bus Stop,Doner Restaurant,Bank,Organic Grocery,Mobile Phone Shop,Restaurant,Sporting Goods Shop,Drugstore
54,80807,1,1,Bus Stop,Museum,Supermarket,Bakery,German Restaurant,Metro Station,Restaurant,Gift Shop,Lounge,Bistro
55,80809,1,1,Bus Stop,Museum,Supermarket,Bakery,German Restaurant,Metro Station,Restaurant,Gift Shop,Lounge,Bistro
56,80937,1,1,Bus Stop,Museum,Supermarket,Bakery,German Restaurant,Metro Station,Restaurant,Gift Shop,Lounge,Bistro


### Cluster 3

In [70]:
muc_merged.loc[muc_merged['Cluster Labels00'] == 2, muc_merged.columns[[1] + list(range(5, muc_merged.shape[1]))]]

,Postal Code,Cluster Labels01,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,80933,2,2,Bus Stop,Supermarket,Indian Restaurant,Pizza Place,Gastropub,Business Service,Food & Drink Shop,Beer Garden,Bakery,Greek Restaurant
33,80935,2,2,Bus Stop,Supermarket,Indian Restaurant,Pizza Place,Gastropub,Business Service,Food & Drink Shop,Beer Garden,Bakery,Greek Restaurant
34,80995,2,2,Bus Stop,Supermarket,Indian Restaurant,Pizza Place,Gastropub,Business Service,Food & Drink Shop,Beer Garden,Bakery,Greek Restaurant


### Cluster 4

In [71]:
muc_merged.loc[muc_merged['Cluster Labels00'] == 3, muc_merged.columns[[1] + list(range(5, muc_merged.shape[1]))]]

,Postal Code,Cluster Labels01,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,80689,3,3,Bus Stop,Bakery,Supermarket,Metro Station,Miscellaneous Shop,German Restaurant,Chinese Restaurant,Drugstore,Greek Restaurant,Residential Building (Apartment / Condo)
36,81375,3,3,Bus Stop,Bakery,Supermarket,Metro Station,Miscellaneous Shop,German Restaurant,Chinese Restaurant,Drugstore,Greek Restaurant,Residential Building (Apartment / Condo)
37,81377,3,3,Bus Stop,Bakery,Supermarket,Metro Station,Miscellaneous Shop,German Restaurant,Chinese Restaurant,Drugstore,Greek Restaurant,Residential Building (Apartment / Condo)


### Cluster 5

In [72]:
muc_merged.loc[muc_merged['Cluster Labels00'] == 4, muc_merged.columns[[1] + list(range(5, muc_merged.shape[1]))]]

,Postal Code,Cluster Labels01,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,80335,4,4,Hotel,Bakery,Doner Restaurant,Bavarian Restaurant,Wine Bar,Drugstore,Ice Cream Shop,Thai Restaurant,Office,Diner
102,80339,4,4,Hotel,Bakery,Doner Restaurant,Bavarian Restaurant,Wine Bar,Drugstore,Ice Cream Shop,Thai Restaurant,Office,Diner


## Result and Discussion 

The result shows that in the Altstadt (old city) of Munich, the most common venues are ice-cream shops, musemums, and Restaurant, which reflects the reality that it is currently one of the most popular tourist place in the city. In cluster2, the most common venues changes from restaurant to Supermarket, which indicates that those are more liveing areas for the citizens. Outer thatn cluster2, the most common venues change to Bus Stop and Hotel, which gives the sign that those places are where most companies are located so there're more visits in Bus Stop (for working) and Hotel (business trips). The analyse around München Haupbahnhof isn't very well as expected though, which could be due to less of information and limited area.
The critical part of the project is where to find the geo data and how to combine different source of data together for further analysis.

## Conclusion 

So now the neighborhoods in Munich are being clustered. For users who want to open a restaurant, it is better to open it in the inner city or near to the living communities, where the potential visits could be higher due to tourism and the citizen people. However, the analysis is only based on single source, there may have other factors which could affect the decision as well.